In [52]:
import numpy as np
import pandas as pd
import requests
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import datetime as dt

In [53]:
symbols_yahoo=["BTC-USD","ETH-USD","USDT-USD","BNB-USD","USDC-USD","BUSD-USD","XRP-USD",
        "ADA-USD","DOGE-USD","MATIC-USD","DOT-USD","DAI-USD","SOL-USD","HEX-USD",
        "TRX-USD","AVAX-USD","LTC-USD","UNI7083-USD",
        "WBTC-USD","LEO-USD","LINK-USD","ATOM-USD", "ETC-USD", "XLM-USD", "CRO-USD",
        "XMR-USD","ALGO-USD", "BCH-USD", "BTCB-USD","NEAR-USD","VET-USD",
        "QNT-USD","FIL-USD","LUNC-USD","CHZ-USD","OKB-USD",
        "HBAR-USD","EGLD-USD","WBNB-USD","XTZ-USD",
        "THETA-USD"]

In [54]:
btc=yf.Ticker("BTC-USD")
btc.info["previousClose"]

17066.676

In [55]:
my_columns=["Ticker","Previous close (USD)","Units to buy","One year return","Six-month return","Three-month return",
           "One-month return","One year return percentile","Six-month return percentile","Three-month return percentile",
           "One-month return percentile"]
qms_first=pd.DataFrame(columns=my_columns)
qms_first["Ticker"]=symbols_yahoo
for i,symbols in zip(qms_first.index,symbols_yahoo):
    qms_first.loc[i,"Previous close (USD)"]=yf.Ticker(symbols).info["previousClose"]
qms_first.head()

,Ticker,Previous close (USD),Units to buy,One year return,Six-month return,Three-month return,One-month return,One year return percentile,Six-month return percentile,Three-month return percentile,One-month return percentile
0,BTC-USD,17066.676,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ETH-USD,1288.1509,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,USDT-USD,0.998284,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BNB-USD,290.36917,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,USDC-USD,1.000699,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
END_DATE=dt.datetime.now()
START_DATE=dt.date(END_DATE.year-2,END_DATE.month,END_DATE.day)
CASHED_DATA=pd.DataFrame()
data_2 = yf.download(' '.join(symbols_yahoo), start=START_DATE, end=END_DATE,
                   group_by='ticker')

tickers_to_retry = []

for ticker in symbols_yahoo:
    download_success = [r for r in data_2[ticker]['Close'] if r > 0]
    if download_success:
        CASHED_DATA[ticker] = {'close': [r for r in data_2[ticker]['Close']],
                               'row_data': data_2[ticker],
                               'low': [r for r in data_2[ticker]['Low']],
                               'high': [r for r in data_2[ticker]['High']]}
    else:
        tickers_to_retry.append(ticker)

if tickers_to_retry:
    data_2 = yf.download(' '.join(tickers_to_retry), start=START_DATE, end=END_DATE,
                       group_by='ticker')

for ticker in tickers_to_retry:
    download_success = [r for r in data_2[ticker]['Close'] if r > 0]
    if download_success:
        CASHED_DATA[ticker] = {'close': [r for r in data_2[ticker]['Close']],
                               'row_data': data_2[ticker],
                               'low': [r for r in data_2[ticker]['Low']],
                               'high': [r for r in data_2[ticker]['High']]}

[*********************100%***********************]  41 of 41 completed


In [57]:
data_portfolio=pd.DataFrame()
for bb in symbols_yahoo:
   data_portfolio[bb]=data_2[bb]["Adj Close"]
data_portfolio.head()

,BTC-USD,ETH-USD,USDT-USD,BNB-USD,USDC-USD,BUSD-USD,XRP-USD,ADA-USD,DOGE-USD,MATIC-USD,...,QNT-USD,FIL-USD,LUNC-USD,CHZ-USD,OKB-USD,HBAR-USD,EGLD-USD,WBNB-USD,XTZ-USD,THETA-USD
Date,,,,,,,,,,,,,,,,,,,,,
2020-11-11,15701.339844,462.960541,1.000985,27.956179,1.000150,1.0001,0.256259,0.105965,0.002769,0.016603,...,14.448093,30.097725,0.320926,0.010748,4.750159,0.032905,7.831029,27.820089,2.121739,0.612423
2020-11-12,16276.343750,461.005280,1.000509,27.614807,0.999207,1.0001,0.254903,0.104525,0.002779,0.015860,...,15.056083,29.517569,0.316530,0.010519,4.820160,0.031096,7.500831,27.488077,2.072694,0.601923
2020-11-13,16317.808594,474.626434,1.000345,28.182173,1.000012,0.9999,0.265191,0.106391,0.002821,0.016917,...,15.475160,29.223934,0.318922,0.010942,4.869129,0.032272,7.767100,28.167414,2.150779,0.633394
2020-11-14,16068.138672,460.149841,1.000288,27.936201,1.000224,0.9997,0.268342,0.104202,0.002773,0.016751,...,15.782834,29.353889,0.305296,0.010731,4.724824,0.031880,7.503135,27.935883,2.100358,0.611983
2020-11-15,15955.587891,447.559082,1.000647,27.621031,1.000168,1.0000,0.269642,0.100531,0.002775,0.016461,...,15.458717,27.825686,0.308027,0.010624,4.647420,0.031270,7.583277,27.509007,2.019590,0.603587


In [58]:
# One year return loop
one_year_ago=dt.date(END_DATE.year-1,END_DATE.month,END_DATE.day)
for i,symbol in zip(qms_first.index,symbols_yahoo):
    qms_first.loc[i,"One year return"]=(yf.Ticker(symbol).info["previousClose"]-data_portfolio.loc["2021-11-11",symbol])/data_portfolio.loc["2021-11-11",symbol]
qms_first.head()

,Ticker,Previous close (USD),Units to buy,One year return,Six-month return,Three-month return,One-month return,One year return percentile,Six-month return percentile,Three-month return percentile,One-month return percentile
0,BTC-USD,17066.676,NaN,-0.737233,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ETH-USD,1288.1509,NaN,-0.727686,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,USDT-USD,0.998284,NaN,-0.002359,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BNB-USD,290.36917,NaN,-0.539018,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,USDC-USD,1.000699,NaN,0.000844,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
# Six-month return loop
for i,symbol in zip(qms_first.index,symbols_yahoo):
    qms_first.loc[i,"Six-month return"]=(qms_first.loc[i,"Previous close (USD)"]-data_portfolio.loc["2022-5-11",symbol])/data_portfolio.loc["2022-5-11",symbol]

# Three-month return loop
for i,symbol in zip(qms_first.index,symbols_yahoo):
    qms_first.loc[i,"Three-month return"]=(qms_first.loc[i,"Previous close (USD)"]-data_portfolio.loc["2022-8-11",symbol])/data_portfolio.loc["2022-8-11",symbol]

# One-month return loop
for i,symbol in zip(qms_first.index,symbols_yahoo):
    qms_first.loc[i,"One-month return"]=(qms_first.loc[i,"Previous close (USD)"]-data_portfolio.loc["2022-10-11",symbol])/data_portfolio.loc["2022-10-11",symbol]
qms_first.head()

,Ticker,Previous close (USD),Units to buy,One year return,Six-month return,Three-month return,One-month return,One year return percentile,Six-month return percentile,Three-month return percentile,One-month return percentile
0,BTC-USD,17066.676,NaN,-0.737233,-0.4102,-0.287628,-0.104178,NaN,NaN,NaN,NaN
1,ETH-USD,1288.1509,NaN,-0.727686,-0.378338,-0.315259,0.006702,NaN,NaN,NaN,NaN
2,USDT-USD,0.998284,NaN,-0.002359,0.002422,-0.002121,-0.001617,NaN,NaN,NaN,NaN
3,BNB-USD,290.36917,NaN,-0.539018,0.07559,-0.103246,0.070007,NaN,NaN,NaN,NaN
4,USDC-USD,1.000699,NaN,0.000844,-0.000067,0.000785,0.000674,NaN,NaN,NaN,NaN


In [63]:
# Llenar percentiles
periodos=["One year","Six-month","Three-month","One-month"]
qms_first["QMS score"]=""
for indice in qms_first.index:
    for per,sym in zip(periodos,symbols_yahoo):
        percentil=stats.percentileofscore(qms_first[f"{per} return"],qms_first.loc[indice,f"{per} return"])
        qms_first.loc[indice,f"{per} return percentile"]=percentil
qms_first.head()

,Ticker,Previous close (USD),Units to buy,One year return,Six-month return,Three-month return,One-month return,One year return percentile,Six-month return percentile,Three-month return percentile,One-month return percentile,QMS score
0,BTC-USD,17066.676,NaN,-0.737233,-0.4102,-0.287628,-0.104178,56.097561,26.829268,46.341463,39.02439,
1,ETH-USD,1288.1509,NaN,-0.727686,-0.378338,-0.315259,0.006702,58.536585,36.585366,36.585366,78.04878,
2,USDT-USD,0.998284,NaN,-0.002359,0.002422,-0.002121,-0.001617,92.682927,73.170732,78.04878,68.292683,
3,BNB-USD,290.36917,NaN,-0.539018,0.07559,-0.103246,0.070007,78.04878,82.926829,68.292683,87.804878,
4,USDC-USD,1.000699,NaN,0.000844,-0.000067,0.000785,0.000674,97.560976,68.292683,80.487805,70.731707,


In [66]:
for indice in qms_first.index:
    media=np.mean([qms_first.loc[indice,"One year return percentile"],
                 qms_first.loc[indice,"Six-month return percentile"],
                 qms_first.loc[indice,"Three-month return percentile"],
                 qms_first.loc[indice,"One-month return percentile"]])
    qms_first.loc[indice,"QMS score"]=media
qms_first.head()
qms_first.sort_values("QMS score",ascending=False,inplace=True)

,Ticker,Previous close (USD),Units to buy,One year return,Six-month return,Three-month return,One-month return,One year return percentile,Six-month return percentile,Three-month return percentile,One-month return percentile,QMS score
9,MATIC-USD,1.063584,NaN,-0.395533,0.596276,0.154308,0.345383,85.365854,92.682927,92.682927,97.560976,92.073171
35,OKB-USD,19.38225,NaN,-0.320885,0.735562,0.038769,0.248606,87.804878,95.121951,90.243902,95.121951,92.073171
34,CHZ-USD,0.22733,NaN,-0.558498,1.136455,0.525742,0.169571,73.170732,100.0,97.560976,90.243902,90.243902
8,DOGE-USD,0.085101,NaN,-0.674151,0.006148,0.199789,0.412278,63.414634,75.609756,95.121951,100.0,83.536585
11,DAI-USD,1.001617,NaN,-0.003717,0.000343,0.002064,0.001928,90.243902,70.731707,85.365854,75.609756,80.487805
4,USDC-USD,1.000699,NaN,0.000844,-0.000067,0.000785,0.000674,97.560976,68.292683,80.487805,70.731707,79.268293
5,BUSD-USD,1.001425,NaN,0.000092,-0.001551,0.001578,0.001522,95.121951,65.853659,82.926829,73.170732,79.268293
3,BNB-USD,290.36917,NaN,-0.539018,0.07559,-0.103246,0.070007,78.04878,82.926829,68.292683,87.804878,79.268293
2,USDT-USD,0.998284,NaN,-0.002359,0.002422,-0.002121,-0.001617,92.682927,73.170732,78.04878,68.292683,78.04878
38,WBNB-USD,289.78833,NaN,-0.539081,0.075336,-0.103991,0.06904,75.609756,80.487805,65.853659,85.365854,76.829268


In [73]:
qms_top_20=qms_first[:20]

In [74]:
portfolio_size=float(input("Ingrese la cantidad USD: "))
position_size = portfolio_size / len(qms_top_20.index)
for h in qms_top_20.index:
    qms_top_20.loc[h, 'Units to buy'] = position_size / qms_top_20['Previous close (USD)'][h]
qms_top_20

Ingrese la cantidad USD: 5000


,Ticker,Previous close (USD),Units to buy,One year return,Six-month return,Three-month return,One-month return,One year return percentile,Six-month return percentile,Three-month return percentile,One-month return percentile,QMS score
9,MATIC-USD,1.063584,235.054285,-0.395533,0.596276,0.154308,0.345383,85.365854,92.682927,92.682927,97.560976,92.073171
35,OKB-USD,19.38225,12.898399,-0.320885,0.735562,0.038769,0.248606,87.804878,95.121951,90.243902,95.121951,92.073171
34,CHZ-USD,0.22733,1099.72524,-0.558498,1.136455,0.525742,0.169571,73.170732,100.0,97.560976,90.243902,90.243902
8,DOGE-USD,0.085101,2937.684773,-0.674151,0.006148,0.199789,0.412278,63.414634,75.609756,95.121951,100.0,83.536585
11,DAI-USD,1.001617,249.596328,-0.003717,0.000343,0.002064,0.001928,90.243902,70.731707,85.365854,75.609756,80.487805
4,USDC-USD,1.000699,249.825272,0.000844,-0.000067,0.000785,0.000674,97.560976,68.292683,80.487805,70.731707,79.268293
5,BUSD-USD,1.001425,249.644157,0.000092,-0.001551,0.001578,0.001522,95.121951,65.853659,82.926829,73.170732,79.268293
3,BNB-USD,290.36917,0.860973,-0.539018,0.07559,-0.103246,0.070007,78.04878,82.926829,68.292683,87.804878,79.268293
2,USDT-USD,0.998284,250.429803,-0.002359,0.002422,-0.002121,-0.001617,92.682927,73.170732,78.04878,68.292683,78.04878
38,WBNB-USD,289.78833,0.862699,-0.539081,0.075336,-0.103991,0.06904,75.609756,80.487805,65.853659,85.365854,76.829268


In [77]:
data_2=pd.DataFrame()
top_symbols=qms_top_20["Ticker"]
END_DATE=dt.datetime.now()
START_DATE=dt.date(END_DATE.year-2,END_DATE.month,END_DATE.day)
CASHED_DATA=pd.DataFrame()
data_2 = yf.download(' '.join(top_symbols), start=START_DATE, end=END_DATE,
                   group_by='ticker')

tickers_to_retry = []

for ticker in top_symbols:
    download_success = [r for r in data_2[ticker]['Close'] if r > 0]
    if download_success:
        CASHED_DATA[ticker] = {'close': [r for r in data_2[ticker]['Close']],
                               'row_data': data_2[ticker],
                               'low': [r for r in data_2[ticker]['Low']],
                               'high': [r for r in data_2[ticker]['High']]}
    else:
        tickers_to_retry.append(ticker)

if tickers_to_retry:
    data_2 = yf.download(' '.join(tickers_to_retry), start=START_DATE, end=END_DATE,
                       group_by='ticker')

for ticker in tickers_to_retry:
    download_success = [r for r in data_2[ticker]['Close'] if r > 0]
    if download_success:
        CASHED_DATA[ticker] = {'close': [r for r in data_2[ticker]['Close']],
                               'row_data': data_2[ticker],
                               'low': [r for r in data_2[ticker]['Low']],
                               'high': [r for r in data_2[ticker]['High']]}

[*********************100%***********************]  20 of 20 completed


In [78]:
data_portfolio=pd.DataFrame()
for bb in top_symbols:
   data_portfolio[bb]=data_2[bb]["Adj Close"]
data_portfolio.head()

,MATIC-USD,OKB-USD,CHZ-USD,DOGE-USD,DAI-USD,USDC-USD,BUSD-USD,BNB-USD,USDT-USD,WBNB-USD,ATOM-USD,LTC-USD,LEO-USD,QNT-USD,UNI7083-USD,LINK-USD,TRX-USD,XMR-USD,XRP-USD,ETH-USD
Date,,,,,,,,,,,,,,,,,,,,
2020-11-11,0.016603,4.750159,0.010748,0.002769,1.007489,1.000150,1.0001,27.956179,1.000985,27.820089,5.033868,59.515030,1.301024,14.448093,2.912375,12.807538,0.025054,113.365768,0.256259,462.960541
2020-11-12,0.015860,4.820160,0.010519,0.002779,1.005264,0.999207,1.0001,27.614807,1.000509,27.488077,5.035270,60.365379,1.291265,15.056083,2.930142,12.423223,0.024937,112.046997,0.254903,461.005280
2020-11-13,0.016917,4.869129,0.010942,0.002821,1.004716,1.000012,0.9999,28.182173,1.000345,28.167414,5.210965,65.457199,1.288315,15.475160,3.879659,12.887978,0.025405,114.112709,0.265191,474.626434
2020-11-14,0.016751,4.724824,0.010731,0.002773,1.003079,1.000224,0.9997,27.936201,1.000288,27.935883,5.047429,63.774151,1.270821,15.782834,3.809107,12.576960,0.025191,116.577446,0.268342,460.149841
2020-11-15,0.016461,4.647420,0.010624,0.002775,1.002197,1.000168,1.0000,27.621031,1.000647,27.509007,4.980141,62.279922,1.271187,15.458717,3.862007,12.121813,0.025409,114.620941,0.269642,447.559082


In [80]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
mu=expected_returns.mean_historical_return(data_portfolio)
s=risk_models.sample_cov(data_portfolio)

In [81]:
ef=EfficientFrontier(mu,s)
weights=ef.max_sharpe()
cleaned_weights=ef.clean_weights()
ef.portfolio_performance(verbose=True)
print(cleaned_weights)

Expected annual return: 220.3%
Annual volatility: 90.7%
Sharpe Ratio: 2.41
OrderedDict([('MATIC-USD', 0.55485), ('OKB-USD', 0.0), ('CHZ-USD', 0.06799), ('DOGE-USD', 0.07123), ('DAI-USD', 0.0), ('USDC-USD', 0.0), ('BUSD-USD', 0.0), ('BNB-USD', 0.0), ('USDT-USD', 0.0), ('WBNB-USD', 0.0), ('ATOM-USD', 0.0), ('LTC-USD', 0.0), ('LEO-USD', 0.30274), ('QNT-USD', 0.0032), ('UNI7083-USD', 0.0), ('LINK-USD', 0.0), ('TRX-USD', 0.0), ('XMR-USD', 0.0), ('XRP-USD', 0.0), ('ETH-USD', 0.0)])


In [82]:
data_port_return=data_portfolio.pct_change()
data_port_return.dropna(inplace=True)
data_port_return.head()

,MATIC-USD,OKB-USD,CHZ-USD,DOGE-USD,DAI-USD,USDC-USD,BUSD-USD,BNB-USD,USDT-USD,WBNB-USD,ATOM-USD,LTC-USD,LEO-USD,QNT-USD,UNI7083-USD,LINK-USD,TRX-USD,XMR-USD,XRP-USD,ETH-USD
Date,,,,,,,,,,,,,,,,,,,,
2020-11-12,-0.044751,0.014737,-0.021306,0.003611,-0.002208,-0.000943,0.0000,-0.012211,-0.000476,-0.011934,0.000279,0.014288,-0.007501,0.042081,0.006101,-0.030007,-0.004670,-0.011633,-0.005292,-0.004223
2020-11-13,0.066646,0.010159,0.040213,0.015113,-0.000545,0.000806,-0.0002,0.020546,-0.000164,0.024714,0.034893,0.084350,-0.002285,0.027834,0.324052,0.037410,0.018767,0.018436,0.040360,0.029547
2020-11-14,-0.009813,-0.029637,-0.019283,-0.017015,-0.001629,0.000212,-0.0002,-0.008728,-0.000057,-0.008220,-0.031383,-0.025712,-0.013579,0.019882,-0.018185,-0.024132,-0.008424,0.021599,0.011882,-0.030501
2020-11-15,-0.017312,-0.016382,-0.009971,0.000721,-0.000879,-0.000056,0.0003,-0.011282,0.000359,-0.015281,-0.013331,-0.023430,0.000288,-0.020536,0.013888,-0.036189,0.008654,-0.016783,0.004845,-0.027362
2020-11-16,0.009052,-0.003279,0.032944,0.040000,-0.000570,-0.000006,-0.0001,0.016041,-0.000235,0.016212,0.024176,0.145253,-0.011293,0.094467,-0.066803,0.038689,0.007360,0.029327,0.058885,0.027664


In [83]:
data_port_return.cov()

,MATIC-USD,OKB-USD,CHZ-USD,DOGE-USD,DAI-USD,USDC-USD,BUSD-USD,BNB-USD,USDT-USD,WBNB-USD,ATOM-USD,LTC-USD,LEO-USD,QNT-USD,UNI7083-USD,LINK-USD,TRX-USD,XMR-USD,XRP-USD,ETH-USD
MATIC-USD,0.007764,2.636783e-03,3.452126e-03,0.002322,1.879373e-05,-1.718517e-06,-3.217267e-06,3.288420e-03,2.527147e-06,3.204787e-03,3.592145e-03,2.923998e-03,1.394751e-04,0.002194,0.003417,0.003573,0.002536,2.377056e-03,0.003114,2.942982e-03
OKB-USD,0.002637,5.053888e-03,2.093393e-03,0.001640,1.023373e-05,-1.652262e-06,-8.434219e-07,2.923145e-03,1.604445e-06,2.870414e-03,1.936170e-03,1.875660e-03,3.963195e-04,0.001391,0.001762,0.002171,0.001543,1.621538e-03,0.001956,1.731031e-03
CHZ-USD,0.003452,2.093393e-03,8.429014e-03,0.002790,1.775026e-05,-1.735390e-06,-2.481814e-06,2.557263e-03,9.798370e-07,2.467690e-03,2.660787e-03,2.506774e-03,2.107666e-04,0.001813,0.002569,0.002676,0.001946,2.042183e-03,0.002171,2.093620e-03
DOGE-USD,0.002322,1.640204e-03,2.789506e-03,0.026155,3.142463e-06,-5.819213e-06,-6.400371e-06,1.827533e-03,-4.553071e-06,1.664942e-03,2.879044e-03,2.796851e-03,-4.553107e-06,0.002821,0.001876,0.002995,0.002200,2.335093e-03,0.002373,2.357727e-03
DAI-USD,0.000019,1.023373e-05,1.775026e-05,0.000003,2.936907e-06,3.818284e-07,3.685621e-07,1.326468e-05,3.853594e-07,1.138273e-05,1.769712e-05,1.275589e-05,-9.330303e-07,0.000017,0.000014,0.000016,0.000011,1.062895e-05,0.000010,1.431397e-05
USDC-USD,-0.000002,-1.652262e-06,-1.735390e-06,-0.000006,3.818284e-07,5.053297e-07,4.236222e-07,3.131106e-07,4.111128e-07,-6.132176e-07,9.133338e-07,-6.474301e-07,-1.482769e-06,0.000002,-0.000001,-0.000001,-0.000002,1.591599e-06,-0.000005,-2.001023e-06
BUSD-USD,-0.000003,-8.434219e-07,-2.481814e-06,-0.000006,3.685621e-07,4.236222e-07,9.054640e-07,-7.652486e-07,3.951099e-07,-1.196843e-06,1.196445e-06,-1.550700e-06,9.781169e-07,0.000003,-0.000003,-0.000001,-0.000003,3.409391e-07,-0.000005,-2.830816e-06
BNB-USD,0.003288,2.923145e-03,2.557263e-03,0.001828,1.326468e-05,3.131106e-07,-7.652486e-07,3.774508e-03,1.867607e-06,3.675281e-03,2.351257e-03,2.161743e-03,3.132436e-04,0.001605,0.002209,0.002453,0.001898,1.986395e-03,0.002242,2.063927e-03
USDT-USD,0.000003,1.604445e-06,9.798370e-07,-0.000005,3.853594e-07,4.111128e-07,3.951099e-07,1.867607e-06,5.971902e-07,1.235633e-06,4.650441e-06,8.023975e-07,-1.961175e-07,0.000006,0.000004,0.000002,-0.000001,8.774804e-07,-0.000001,6.546790e-07
WBNB-USD,0.003205,2.870414e-03,2.467690e-03,0.001665,1.138273e-05,-6.132176e-07,-1.196843e-06,3.675281e-03,1.235633e-06,3.800641e-03,2.176770e-03,2.089595e-03,2.927263e-04,0.001476,0.002133,0.002372,0.001860,1.807049e-03,0.002174,2.008349e-03
